In [1]:
from datetime import datetime
from elasticsearch import Elasticsearch
import numpy as np
import random
import time
from urllib.parse import urlparse
from scipy import sparse
import math
import time
from math import log2,pow

In [2]:
import elasticsearch
import elasticsearch_dsl

In [3]:
es = Elasticsearch()

In [4]:
crawled = {}
results = elasticsearch.helpers.scan(es,
    index="crawled_data_final",
    doc_type="doc",
    preserve_order=True,
    query={"query": {"match_all": {}}},
)

# for item in results:
#     crawled[item['_id']] = True

In [5]:
for item in results:
    crawled[item['_id']] = item['_source']

In [6]:
len(crawled)

39955

In [7]:
urls = []
st = time.time()
for link in crawled:
    urls.append(link)
    for url in crawled[link]['INLINK']:
        urls.append(url)
    for url in crawled[link]['OUTLINK']:
        urls.append(url)

#     urls.extend(crawled[link]['INLINK'])
#     urls.extend(crawled[link]['OUTLINK'])
print(time.time() - st)

    

0.37928152084350586


In [8]:
urls = list(set(urls))

In [11]:
i = 0
graph = {}

_set = []
for link in crawled:
    if link not in graph:
        graph[link] = {
            "inlinks" : [],
            "outlinks": []
        }

#     if link not in crawled:
#         continue
        
    outlink = []
    
    for ol in crawled[link]["OUTLINK"]:
        if ol in crawled:
            outlink.append(ol)
        else:
            o = urlparse(ol)
            outlink.append(o.scheme + "://" + o.netloc)
    
#     outlink = list(set(outlink))
#     _set.append(link)
#     _set.extend(outlink)

    graph[link]["outlinks"] = list(set(outlink))
    
#     print(len(outlink), len(graph[link]["outlinks"]), end=", ")
    
    for in_l in graph[link]["outlinks"]:
        if in_l not in graph:
            graph[in_l] = {
                "inlinks" : [],
                "outlinks": []
            }           
        graph[in_l]["inlinks"].append(link)
#         _set.append(in_l)

    if i % 1000 == 0:
        print(i,end =" ")
    i = i + 1
            
    
        
        

0 1000 2000 3000 4000 5000 6000 7000 8000 9000 10000 11000 12000 13000 14000 15000 16000 17000 18000 19000 20000 21000 22000 23000 24000 25000 26000 27000 28000 29000 30000 31000 32000 33000 34000 35000 36000 37000 38000 39000 

In [12]:
len(graph)

53617

In [22]:
for node in graph:
    graph[node]["inlinks"] = list(set(graph[node]["inlinks"]))
    graph[node]["outlinks"] = list(set(graph[node]["outlinks"]))

In [23]:
pr = {}

In [24]:
t = len(graph)
for link in graph:
    pr[link] = 1 / t

In [25]:
sink_nodes = {}

In [26]:
for link in graph:
    if len(graph[link]["outlinks"]) <= 0:
        sink_nodes[link] = True

In [27]:
t - len(sink_nodes)

35580

In [28]:
def perpex(pr):
    ent = 0
    for link in pr:
        ent = ent + (pr[link] * log2( 1 / pr[link]))
    return pow(2,ent)
    

In [29]:
cond = True
d = 0.85
plex = []

In [30]:
x = 1
perval = perpex(pr)
print(x, perval)


while cond:
    new_pr = {}
    sinkpr = 0 
    for link in sink_nodes:
        sinkpr = sinkpr + pr[link]
        
    for link in pr:
        new_pr[link] = (1 - d) / t
        new_pr[link] = new_pr[link] + (d * (sinkpr / t))
        
        for inlinkto in graph[link]['inlinks']:
            new_pr[link] = new_pr[link] + (d * pr[inlinkto] / len(graph[inlinkto]["outlinks"]))

    for page in pr:
        pr[page] = new_pr[page]
        
    perval = perpex(pr)
    x = x + 1
    print(x, perval)
    plex.append(perval)
    
    if(len(plex) == 4):
        if( int(plex[0]) % 10 == int(plex[1]) % 10 and 
            int(plex[1]) % 10 == int(plex[2]) % 10 and
            int(plex[2]) % 10 == int(plex[3]) % 10):
            cond = False
        else:
            del plex[0]
    
    
    if(x % 100 == 0):
        print(x)
        print(time.time() - st)
        st = time.time()


#     break


1 53617.00000080611
2 23536.067356317944
3 24946.334309225764
4 23564.448990613466
5 23995.632387892067
6 23555.865528978793
7 23825.379014479928
8 23561.633985563803
9 23729.38432227564
10 23550.642290031967
11 23658.515038068803
12 23534.07500433427
13 23605.316252821434
14 23518.247299521856
15 23566.105108218755
16 23505.162185331403
17 23537.70434535216
18 23495.050392967
19 23517.369163422438
20 23487.51903576875
21 23502.917839751168
22 23482.025605984258
23 23492.69697280004
24 23478.069007679416
25 23485.488367017617
26 23475.241656995375
27 23480.41232375946
28 23473.230899222217
29 23476.840899499628
30 23471.805004833463
31 23474.328865170137
32 23470.795552762927
33 23472.561937753122
34 23470.08153920389
35 23471.318830333614
36 23469.57667093091
37 23470.443942667443
38 23469.219686411132
39 23469.827938058665
40 23468.967210819246
41 23469.394004424896
42 23468.788581748027
43 23469.088173847595
44 23468.662140896715
45 23468.872518157288
46 23468.572594214176
47 23468.

In [31]:
sorted(pr.items(), key=lambda kv: kv[1])[::-1][0:60]

[('https://en.wikipedia.org', 0.01017879659465084),
 ('http://en.wikipedia.org', 0.009076257710721115),
 ('http://en.m.wikipedia.org', 0.007747637710965084),
 ('https://commons.wikimedia.org', 0.00332002897171911),
 ('http://www.facebook.com', 0.002472985522425136),
 ('http://twitter.com/SaratogaChamber', 0.0021464684551627186),
 ('http://www.youtube.com/user/SaratogaTourism', 0.0018941840587644657),
 ('https://commons.wikimedia.org/wiki/Category:George_Washington_Memorial_Parkway_(HAER)',
  0.00179635254559802),
 ('https://www.instagram.com', 0.0017593659520925673),
 ('https://www.google.com', 0.0017508551265426459),
 ('https://commons.wikimedia.org/wiki/Category:George_Washington_Memorial_Parkway',
  0.0017459615070151923),
 ('https://commons.wikimedia.org/wiki/Category:Printers_marks_and_columns_in_the_Thomas_Jefferson_Building',
  0.001696806821548303),
 ('http://twitter.com/hashtag/ilovesaratoga', 0.0016573605105723883),
 ('https://www.pinterest.com/saratogatourism/', 0.0016417601

In [ ]:
#  in crawled["'https://en.wikipedia.org/wiki/Parliament_of_the_Bahamas'"]

In [ ]:
# graph["http://www.bahamas.gov.bs/wps/portal/public/The%20Government/Legislative/About%20the%20Parliament/!ut/p/b1/vZLLbqMwGEafpQ-QYsx9SYAYk2ACGAJsEJBwCbkTQsjTTzoaaWY0aruZ1l5ZOvan7_xmEiZikkN2a6rs2hwP2e7tnIgpB5CtqrxsIwGIAAfLhbqUTOiwwhOInwB4Z6ngw_tzyKyYKIrhXT_iYWaEaZ2FW3BPsmq070Ow2lZD150auAwq-1r5ZZnzzqLIL5xgSW13yljSa7YKO3xTpLMwtqjGmjERvEVF3I0K6aHXVpnSr7gUylFh8mtSHok2V-wSOcjPD5ekpOJZ0oGz6kwze2zqBHYSW-nYOG01QB4WbcKpwlIudpqe1lnsdqedNevmuzSc3jWXa8HLLwfvleTBZw5XTPIh8vbCT-AjzZ-JJuZxv2HiJyb9gdEQAMwC1fNZF8iEZSgTAT71t-MJP9qH9"]

In [ ]:
# crawled["http://www.bahamas.gov.bs/wps/portal/public/The%20Government/Legislative/About%20the%20Parliament/!ut/p/b1/vZLLbqMwGEafpQ-QYsx9SYAYk2ACGAJsEJBwCbkTQsjTTzoaaWY0aruZ1l5ZOvan7_xmEiZikkN2a6rs2hwP2e7tnIgpB5CtqrxsIwGIAAfLhbqUTOiwwhOInwB4Z6ngw_tzyKyYKIrhXT_iYWaEaZ2FW3BPsmq070Ow2lZD150auAwq-1r5ZZnzzqLIL5xgSW13yljSa7YKO3xTpLMwtqjGmjERvEVF3I0K6aHXVpnSr7gUylFh8mtSHok2V-wSOcjPD5ekpOJZ0oGz6kwze2zqBHYSW-nYOG01QB4WbcKpwlIudpqe1lnsdqedNevmuzSc3jWXa8HLLwfvleTBZw5XTPIh8vbCT-AjzZ-JJuZxv2HiJyb9gdEQAMwC1fNZF8iEZSgTAT71t-MJP9qH9"]